<a href="https://colab.research.google.com/github/bhagyapatel/Big-Data/blob/main/json_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !apt-get install openjdk-8-jdk-headless -qq > /dev/null
 !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
 !tar xf spark-3.0.0-bin-hadoop3.2.tgz
 !pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ["path"] ="${SPARK_HOME}/bin"

In [3]:
!pip install -q findspark
import findspark
findspark.init()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("json_File").getOrCreate()
from pyspark import SparkFiles,SparkContext

In [5]:
#url = "https://github.com/spark-examples/spark-scala-examples/blob/master/src/main/resources/multiline-zipcode.json"
url5 = "https://raw.githubusercontent.com/azar-s91/dataset/master/ford_json.json"
spark.sparkContext.addFile(url5)


In [ ]:
ford_df = spark.read.option("multipline",True).json("file://" +SparkFiles.get("ford_json.json"))


In [ ]:
zip_url = "https://raw.githubusercontent.com/spark-examples/spark-scala-examples/master/src/main/resources/zipcodes.json"
spark.sparkContext.addFile(zip_url)
zip_df = spark.read.json("file://" +SparkFiles.get("zipcodes.json"))

In [ ]:
zip_df.show()

+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|               City|Country|Decommisioned|EstimatedPopulation|  Lat|            Location|        LocationText|  LocationType|   Long|        Notes|RecordNumber|State|TaxReturnsFiled|TotalWages|WorldRegion|Xaxis|Yaxis|Zaxis|ZipCodeType|Zipcode|
+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|        PARC PARQUE|     US|        false|               null|17.96|NA-US-PR-PARC PARQUE|     Parc Parque, PR|NOT ACCEPTABLE| -66.22|         null|           1|   PR|           null|      null|         NA| 0.38|-0.87|  0.3|   STANDARD|    704|
|PASEO COSTA DEL SUR

In [ ]:
zip_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- EstimatedPopulation: long (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Long: double (nullable = true)
 |-- Notes: string (nullable = true)
 |-- RecordNumber: long (nullable = true)
 |-- State: string (nullable = true)
 |-- TaxReturnsFiled: long (nullable = true)
 |-- TotalWages: long (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Xaxis: double (nullable = true)
 |-- Yaxis: double (nullable = true)
 |-- Zaxis: double (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- Zipcode: long (nullable = true)



convert dictionary into dataframe

In [ ]:
from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

jsonString= """{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+



In [ ]:
#PySpark from_json() function is used to convert JSON string into Struct type or Map type.
from pyspark.sql.types import MapType,StringType
df2=df.withColumn("value",from_json(col("value"),MapType(StringType(),StringType())))


In [ ]:
df2.show()

In [ ]:
#to_json() function is used to convert DataFrame columns MapType or Struct type to JSON string.
from pyspark.sql.functions import to_json,col
df2.withColumn("value",to_json(col("value"))) \
   .show(truncate=False)

+---+----------------------------------------------------------------------------+
|id |value                                                                       |
+---+----------------------------------------------------------------------------+
|1  |{"Zipcode":"704","ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+----------------------------------------------------------------------------+



In [ ]:
#Function json_tuple() is used the query or extract the elements from JSON column and create the result as a new columns.
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")) \
    .toDF("id","Zipcode","ZipCodeType","City") \
    .show(truncate=False)

+---+-------+-----------+-----------+
|id |Zipcode|ZipCodeType|City       |
+---+-------+-----------+-----------+
|1  |704    |STANDARD   |PARC PARQUE|
+---+-------+-----------+-----------+



In [ ]:
#column = ["id","Zipcode","ZipCodeType","City","State"]
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City","State")).toDF("id","Zipcode","ZipCodeType","City","State").show()

+---+-------+-----------+-----------+-----+
| id|Zipcode|ZipCodeType|       City|State|
+---+-------+-----------+-----------+-----+
|  1|    704|   STANDARD|PARC PARQUE|   PR|
+---+-------+-----------+-----------+-----+



In [ ]:
dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'red','eye':'grey'}),
        ('Jefferson',{'hair':'red','eye':''})
        ]
df = spark.createDataFrame(dataDictionary, schema = ["name","properties"])

df.show()
column = ["name","eye","hair"]
df1 = df.rdd.map(lambda x: (x.name, x.properties["eye"],x.properties["hair"])).toDF(column)


+----------+--------------------+
|      name|          properties|
+----------+--------------------+
|     James|[eye -> brown, ha...|
|   Michael|[eye ->, hair -> ...|
|    Robert|[eye -> black, ha...|
|Washington|[eye -> grey, hai...|
| Jefferson|[eye -> , hair ->...|
+----------+--------------------+



In [ ]:
type(jsonString)

str

Multiple line json:-

In [ ]:
multiline_url = "https://raw.githubusercontent.com/spark-examples/spark-scala-examples/master/src/main/resources/multiline-zipcode.json"
spark.sparkContext.addFile(multiline_url)
multi_df = spark.read.option("multiline",True).json("file://"+SparkFiles.get("multiline-zipcode.json"))
multi_df.show()

+-------------------+------------+-----+-----------+-------+
|               City|RecordNumber|State|ZipCodeType|Zipcode|
+-------------------+------------+-----+-----------+-------+
|PASEO COSTA DEL SUR|           2|   PR|   STANDARD|    704|
|       BDA SAN LUIS|          10|   PR|   STANDARD|    709|
+-------------------+------------+-----+-----------+-------+



In [ ]:
#--using this we can run any python file
%run abc.py 

using NLP convert one col into into datafram

In [ ]:
df = spark.read.csv("/content/df1.txt")

In [ ]:
df.show(truncate=False)

+--------------------------------------------------------------------------------------+
|_c0                                                                                   |
+--------------------------------------------------------------------------------------+
|rohan|kumar|pune|9876543456|hina|kumari|us|6789087654|mari|patel|ahmedabad|9876898765"|
+--------------------------------------------------------------------------------------+



In [ ]:
df1 = df.withColumn("chk",regexp_replace("_c0","(.*?\\|){4}","$0-")).drop("_c0")
df1.show(truncate=False)

+----------------------------------------------------------------------------------------+
|chk                                                                                     |
+----------------------------------------------------------------------------------------+
|rohan|kumar|pune|9876543456|-hina|kumari|us|6789087654|-mari|patel|ahmedabad|9876898765"|
+----------------------------------------------------------------------------------------+



In [ ]:
df2 = df1.withColumn("chk", explode(split('chk','\|-')))

In [ ]:
df2.show(truncate=False)

+--------------------------------+
|chk                             |
+--------------------------------+
|rohan|kumar|pune|9876543456     |
|hina|kumari|us|6789087654       |
|mari|patel|ahmedabad|9876898765"|
+--------------------------------+



In [ ]:
column = ["name","lastname","city","no"]
df3 = df2.rdd.map(lambda x: x[0].split("|")).toDF(column)
df3.show()

+-----+--------+---------+-----------+
| name|lastname|     city|         no|
+-----+--------+---------+-----------+
|rohan|   kumar|     pune| 9876543456|
| hina|  kumari|       us| 6789087654|
| mari|   patel|ahmedabad|9876898765"|
+-----+--------+---------+-----------+



dealing with bad records

In [ ]:
#it ignore bad recore 
badrecord1 = spark.read.option("mode","permissive").option("header",True).csv("/content/bad_Records.csv")
badrecord1.show()

In [ ]:
badrecord1.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)



In [ ]:
# this will throw error if it meet with bad record
badrecord = spark.read.option("mode","FAILFAST").option("header",True).csv("/content/bad_Records.csv")
badrecord.show()

In [ ]:
# THIS will skips bad records
badrecord = spark.read.option("mode","DROPMALFORMED").option("header",True).csv("/content/bad_Records.csv")
badrecord.show()

+-------+-------+-----------+
|     id|   name| department|
+-------+-------+-----------+
|      1| murali|   healcare|
|      2| ganesh|   finance |
|invalid|   maya| healthcare|
|      4|   nita|       shop|
|      5|   none|  housewife|
|      6| alpesh|      sales|
+-------+-------+-----------+



In [ ]:
from pyspark.sql.types import *
sch = StructType([
    StructField("id",StringType(),True),
    StructField("name",StringType(),True),
    StructField("department",StringType(),True)
    ])

In [ ]:
badrecord = spark.read.option("mode","DROPMALFORMED").option("header",True).option("schema",sch).csv("/content/bad_Records.csv")


In [ ]:
badrecord.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)



Multiple Delimiter

In [30]:
multiple_del = spark.read.option("header",True).option("delimiter",True).csv("/content/multiple_delimiter.txt")

In [31]:
multiple_del.show(truncate=False)

+--------------------------+
|name ~| age ~| city       |
+--------------------------+
|azahar,ali ~| 45 ~| pune  |
|hina, khan ~| 55 ~| ahme  |
|anmol, patel ~| 70 ~| udna|
|bhagya, patel ~| 30 ~| us |
+--------------------------+



In [33]:
column = ["name", "age", "city"]
df2 = multiple_del.rdd.map(lambda x: x[0].split("~|")).toDF(column)
df2.show()

+--------------+----+-----+
|          name| age| city|
+--------------+----+-----+
|   azahar,ali | 45 | pune|
|   hina, khan | 55 | ahme|
| anmol, patel | 70 | udna|
|bhagya, patel | 30 |   us|
+--------------+----+-----+



In [36]:
df2.withColumn("FirstName", split(df2["name"], ",").getItem(0)) \
.withColumn("lastname", split(df2["name"], ",").getItem(1)).drop("name").show()

+----+-----+---------+--------+
| age| city|FirstName|lastname|
+----+-----+---------+--------+
| 45 | pune|   azahar|    ali |
| 55 | ahme|     hina|   khan |
| 70 | udna|    anmol|  patel |
| 30 |   us|   bhagya|  patel |
+----+-----+---------+--------+



convert date into date format and add no of days in existing dates 

In [100]:
df1 = spark.read.option("header",True).csv("/content/date_df.txt")

In [101]:
df1.withColumn("date", to_date(col("date11"),"yyyyMMdd")).show()

+----+--------+--------+----+----------+
|  id|  date11|    name| no1|      date|
+----+--------+--------+----+----------+
|2004|20221115| bhawana|2004|2022-11-15|
|2003|20221124|    hina|2003|2022-11-24|
|2005|20181020|hirakshi|2002|2018-10-20|
|2006|20170810|  mohini|2001|2017-08-10|
+----+--------+--------+----+----------+



In [109]:
df2 = df1.withColumn("date", to_date(col("date11").cast("String"),"yyyyMMdd"))

In [110]:
df2.select("date").show()

+----------+
|      date|
+----------+
|2022-11-15|
|2022-11-24|
|2018-10-20|
|2017-08-10|
+----------+



In [111]:
df2.withColumn("date", date_add("date",4)).show()

+----+--------+--------+----+----------+
|  id|  date11|    name| no1|      date|
+----+--------+--------+----+----------+
|2004|20221115| bhawana|2004|2022-11-19|
|2003|20221124|    hina|2003|2022-11-28|
|2005|20181020|hirakshi|2002|2018-10-24|
|2006|20170810|  mohini|2001|2017-08-14|
+----+--------+--------+----+----------+



map and flatmap in rdd

In [112]:
rdd1 = spark.sparkContext.textFile("/content/date_df.txt")

In [113]:
rdd1.collect()

['id,date11,name,no1',
 '2004,20221115,bhawana,2004',
 '2003,20221124,hina,2003',
 '2005,20181020,hirakshi,2002',
 '2006,20170810,mohini,2001']

In [117]:
rdd1.count()

5

In [127]:
rdd2 = rdd1.flatMap(lambda x: x.split(","))
rdd2.collect()

['id',
 'date11',
 'name',
 'no1',
 '2004',
 '20221115',
 'bhawana',
 '2004',
 '2003',
 '20221124',
 'hina',
 '2003',
 '2005',
 '20181020',
 'hirakshi',
 '2002',
 '2006',
 '20170810',
 'mohini',
 '2001']

In [128]:
rdd2.count()

20

In [130]:
for i in rdd2.collect():
  print(i)

id
date11
name
no1
2004
20221115
bhawana
2004
2003
20221124
hina
2003
2005
20181020
hirakshi
2002
2006
20170810
mohini
2001


How to use explode and create new col with index

In [132]:
df = spark.read.option("header",True).option("delimiter","|").csv("/content/explod.txt")

In [133]:
df.show()

+---+-----+-----------+
| id| name|  education|
+---+-----+-----------+
|  1| hari|    mba,bba|
|  2| lila|    bca,mca|
|  3| hina|       null|
|  4|naman|bba,mba,phd|
+---+-----+-----------+



In [140]:
df.withColumn("qualification1",explode(split("education",","))).show()

+---+-----+-----------+--------------+
| id| name|  education|qualification1|
+---+-----+-----------+--------------+
|  1| hari|    mba,bba|           mba|
|  1| hari|    mba,bba|           bba|
|  2| lila|    bca,mca|           bca|
|  2| lila|    bca,mca|           mca|
|  4|naman|bba,mba,phd|           bba|
|  4|naman|bba,mba,phd|           mba|
|  4|naman|bba,mba,phd|           phd|
+---+-----+-----------+--------------+



In [139]:
from re import split
from pyspark.sql.functions import *
df.withColumn("qualification1",explode_outer(split("education",","))).show()

+---+-----+-----------+--------------+
| id| name|  education|qualification1|
+---+-----+-----------+--------------+
|  1| hari|    mba,bba|           mba|
|  1| hari|    mba,bba|           bba|
|  2| lila|    bca,mca|           bca|
|  2| lila|    bca,mca|           mca|
|  3| hina|       null|          null|
|  4|naman|bba,mba,phd|           bba|
|  4|naman|bba,mba,phd|           mba|
|  4|naman|bba,mba,phd|           phd|
+---+-----+-----------+--------------+



In [146]:
df.select("*",posexplode_outer(split("education",","))).show()

+---+-----+-----------+----+----+
| id| name|  education| pos| col|
+---+-----+-----------+----+----+
|  1| hari|    mba,bba|   0| mba|
|  1| hari|    mba,bba|   1| bba|
|  2| lila|    bca,mca|   0| bca|
|  2| lila|    bca,mca|   1| mca|
|  3| hina|       null|null|null|
|  4|naman|bba,mba,phd|   0| bba|
|  4|naman|bba,mba,phd|   1| mba|
|  4|naman|bba,mba,phd|   2| phd|
+---+-----+-----------+----+----+



In [149]:
df.select("*",posexplode_outer(split("education",","))) \
.withColumnRenamed("pos","position") \
.withColumnRenamed("col","qualification") \
.drop("education") \
.show()

+---+-----+--------+-------------+
| id| name|position|qualification|
+---+-----+--------+-------------+
|  1| hari|       0|          mba|
|  1| hari|       1|          bba|
|  2| lila|       0|          bca|
|  2| lila|       1|          mca|
|  3| hina|    null|         null|
|  4|naman|       0|          bba|
|  4|naman|       1|          mba|
|  4|naman|       2|          phd|
+---+-----+--------+-------------+

